In [5]:
import os
from langchain.graphs import Neo4jGraph
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import GraphCypherQAChain

In [ ]:
# Set environment variables for OpenAI API key
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

# Configure Neo4j graph connection
neo4j_graph = Neo4jGraph(
    url="bolt://localhost:7687",  # Adjust if hosted elsewhere
    username="neo4j",
    password="neo4j"
)

# Initialize OpenAI model
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
# Example Cypher query: Fetch companies managed by a specific manager
cypher_query = """
MATCH (m:Manager {name: $manager_name})-[:HOLDS]->(c:Company)
RETURN c.name AS company_name, c.cusip6 AS cusip6, c.cusip AS cusip
"""

# Execute the Cypher query with a manager's name
manager_name = "Royal Bank of Canada"
results = neo4j_graph.query(cypher_query, parameters={"manager_name": manager_name})

# Print results
print("Companies managed by:", manager_name)
for result in results:
    print(result["company_name"], "-", result["cusip6"], "-", result["cusip"])

In [ ]:
# Create a GraphCypherQAChain for LLM-powered Cypher querying
graph_qa_chain = GraphCypherQAChain.from_llm(llm, graph=neo4j_graph)

# Example question for the LLM
question = "Which companies does John Doe manage, and what are their CUSIP6 numbers?"

# Use the LLM to interpret the question, run the query, and generate insights
response = graph_qa_chain.run(question)
print(response)